In [28]:
import Pkg; Pkg.add("JSON")

using JSON

   Resolving package versions...


  No Changes to `C:\Users\flama\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\flama\.julia\environments\v1.8\Manifest.toml`


In [29]:
json_toy = JSON.parsefile("toy.json")


Dict{String, Any} with 7 entries:
  "land_substation_cable_types"       => Any[Dict{String, Any}("rating"=>200, "…
  "wind_turbines"                     => Any[Dict{String, Any}("id"=>1, "x"=>80…
  "wind_scenarios"                    => Any[Dict{String, Any}("power_generatio…
  "substation_locations"              => Any[Dict{String, Any}("id"=>1, "x"=>40…
  "substation_types"                  => Any[Dict{String, Any}("cost"=>200.0, "…
  "general_parameters"                => Dict{String, Any}("maximum_curtailing"…
  "substation_substation_cable_types" => Any[Dict{String, Any}("rating"=>100, "…

In [30]:
json_small = JSON.parsefile("small.json")

Dict{String, Any} with 7 entries:
  "land_substation_cable_types"       => Any[Dict{String, Any}("rating"=>200, "…
  "wind_turbines"                     => Any[Dict{String, Any}("id"=>1, "x"=>79…
  "wind_scenarios"                    => Any[Dict{String, Any}("power_generatio…
  "substation_locations"              => Any[Dict{String, Any}("id"=>1, "x"=>35…
  "substation_types"                  => Any[Dict{String, Any}("cost"=>300.0, "…
  "general_parameters"                => Dict{String, Any}("maximum_curtailing"…
  "substation_substation_cable_types" => Any[Dict{String, Any}("rating"=>100, "…

In [31]:
json_medium = JSON.parsefile("medium.json")

Dict{String, Any} with 7 entries:
  "land_substation_cable_types"       => Any[Dict{String, Any}("rating"=>200, "…
  "wind_turbines"                     => Any[Dict{String, Any}("id"=>1, "x"=>78…
  "wind_scenarios"                    => Any[Dict{String, Any}("power_generatio…
  "substation_locations"              => Any[Dict{String, Any}("id"=>1, "x"=>40…
  "substation_types"                  => Any[Dict{String, Any}("cost"=>300.0, "…
  "general_parameters"                => Dict{String, Any}("maximum_curtailing"…
  "substation_substation_cable_types" => Any[Dict{String, Any}("rating"=>100, "…

In [32]:
json_large = JSON.parsefile("large.json")

Dict{String, Any} with 7 entries:
  "land_substation_cable_types"       => Any[Dict{String, Any}("rating"=>200, "…
  "wind_turbines"                     => Any[Dict{String, Any}("id"=>1, "x"=>78…
  "wind_scenarios"                    => Any[Dict{String, Any}("power_generatio…
  "substation_locations"              => Any[Dict{String, Any}("id"=>1, "x"=>65…
  "substation_types"                  => Any[Dict{String, Any}("cost"=>240.0, "…
  "general_parameters"                => Dict{String, Any}("maximum_curtailing"…
  "substation_substation_cable_types" => Any[Dict{String, Any}("rating"=>100, "…

In [33]:
json_huge = JSON.parsefile("huge.json")

Dict{String, Any} with 7 entries:
  "land_substation_cable_types"       => Any[Dict{String, Any}("rating"=>200, "…
  "wind_turbines"                     => Any[Dict{String, Any}("id"=>1, "x"=>79…
  "wind_scenarios"                    => Any[Dict{String, Any}("power_generatio…
  "substation_locations"              => Any[Dict{String, Any}("id"=>1, "x"=>22…
  "substation_types"                  => Any[Dict{String, Any}("cost"=>240.0, "…
  "general_parameters"                => Dict{String, Any}("maximum_curtailing"…
  "substation_substation_cable_types" => Any[Dict{String, Any}("rating"=>100, "…

In [8]:
function get_parameters(name_json)
    general_parameters = name_json["general_parameters"]
    return general_parameters
end

function get_wind_turbines(name_json)
    get_wind_turbines = name_json["wind_turbines"]

    return get_wind_turbines
end

function get_wind_scenarios(name_json)
    get_wind_scenarios = name_json["wind_scenarios"]
    return get_wind_scenarios
end

function get_substation_locations(name_json)
    get_substation_locations = name_json["substation_locations"]
    
    return get_substation_locations
end

function get_substation_types(name_json)
    get_substation_types = name_json["substation_types"]

    return get_substation_types
end

function get_land_substation_cable_types(name_json)
    get_land_substation_cable_types = name_json["land_substation_cable_types"]

    return get_land_substation_cable_types
end

function get_substation_substation_cable_types(name_json)
    get_substation_substation_cable_types = name_json["substation_substation_cable_types"]

    return get_substation_substation_cable_types
end

function cost_turbines_substation(L, D1, general_parameters)
    # L : vecteur liant turbines et sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    for i in 1:length(L)
        cost += general_parameters["C_ft"] +D1[i,L[i]]*general_parameters["C_lt"]
    end

    return cost
end

function cost_substation_substation(S, D2, substation_substation_cable_types)
    # S : vecteur liant les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # substation_substation_cable_types : types de câbles entre les sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations
    for i in 1:length(S)
        cost += substation_substation_cable_types[S[i][3]]["fixed_cost"]+D2[i,S[i][2]]*substation_substation_cable_types[S[i][3]]["variable_cost"]
    end

    cost = cost/2

    return cost
end

function cost_substation_land_substation(S, D3, land_substation_cable_types)
    # S : vecteur liant les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    for i in 1:length(S)
        cost += land_substation_cable_types[S[i][4]]["fixed_cost"]+D3[i]*land_substation_cable_types[S[i][4]]["variable_cost"]
    end
end

function cost_substation(S, substation_types)
    # S : vecteur liant les sous-stations
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de construction des sous-stations
    for i in 1:length(S)
        cost += substation_types[S[i][1]]["cost"]
    end

    return cost
end


function c_c(C, general_parameters)

    return general_parameters["curtailing_cost"]*C + general_parameters["curtailing_penalty"]*max(0,C-general_parameters["maximum_curtailing"])
end


function C_f_aux(v, v1, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f_aux = 0

    C_f_aux += wind_scenarios[w][" power_generation "]*S[v1][1]

    if S[v1][2]==v
        C_f_aux += min(substation_substation_cable_types[S[v1][3]]["rating"], wind_scenarios[w][" power_generation "]*S[v][1])
    else
        C_f_aux += min(0, wind_scenarios[w][" power_generation "]*S[v][1])
    end

    if S[v1][4]!=0
        C_f_aux -= min(substation_types[S[v1][5]],land_substation_cable_types[S[v1][4]]["rating"])
    end
end


function C_f(v, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f = 0

    if S[v][2]!=0
        Cf += max(0, wind_scenarios[w][" power_generation "]*S[v][1]-substation_substation_cable_types[S[v][3]]["rating"])
    else
        Cf += max(0, wind_scenarios[w][" power_generation "]*S[v][1])
    end

    for i in 1:length(S)
        if S[i][4]!=0
            Cf += max(0, C_f_aux(v, i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types))
        end
    end

    return C_f

end


function C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    
    C_n = 0

    for i in 1:length(S)
        if S[i][4]!=0
            C_n += max(0, wind_scenarios[w][" power_generation "]*S[i][1]-min(min(substation_types[S[v1][5]],land_substation_cable_types[S[v1][4]]["rating"])))
        end
    end

    return C_n
end
    

function p_f(v, S, land_substation_cable_types, substation_types)
    p_f = 0

    p_f += land_substation_cable_types[S[v][4]]["probability_of_failure "]

    p_f += substation_types[S[v][5]]["probability_of_failure "]

    return p_f
end


function operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    cost = 0

    for i in 1:length(S)
        cost += p_f(i, S, land_substation_cable_types, substation_types)*c_c(C_f(i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types,general_parameters))
        cost += (1-p_f(i, S, land_substation_cable_types, substation_types))*c_c(C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types,general_parameters))
    end
end

function operational_cost(L, S, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    for w in 1:length(wind_scenarios)
        cost += wind_scenarios[w]["probability"]*operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
        
    end

    return cost
end

function cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème
    # substation_substation_cable_types : types de câbles entre les sous-stations
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    cost += cost_turbines_substation(L, D1, general_parameters)

    # On calcule le coût de la liaison entre les sous-stations
    cost += cost_substation_substation(S, D2, substation_substation_cable_types)

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    cost += cost_substation_land_substation(S, D3, land_substation_cable_types)

    # On calcule le coût de construction des sous-stations
    cost += cost_substation(S, substation_types)

    # On calcule le coût opérationnel
    cost += operational_cost(L, S, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)

    return cost
end
    



function distance(V_t,V_s,v_0)
    # V_t : vecteur des turbines
    # V_s : vecteur des sous-stations
    # v_0 : vecteur de la station terrestre
    
    # On initialise le vecteur des distances
    D1 = zeros(length(V_t),length(V_s))
    D2 = zeros(length(V_s),length(V_s))
    D3 = zeros(length(V_s),1)

    # On calcule les distances entre les turbines et les sous-stations, et entre les sous-stations et la station terrestre et entre les sous-stations
    for i in 1:length(V_t)
        for j in 1:length(V_s)
            D1[i,j] = sqrt((V_t[i][1]-V_s[j][1])^2 + (V_t[i][2]-V_s[j][2])^2)
        end
    end

    for i in 1:length(V_s)
        for j in 1:length(V_s)
            D2[i,j] = sqrt((V_s[i][1]-V_s[j][1])^2 + (V_s[i][2]-V_s[j][2])^2)
        end
    end

    for i in 1:length(V_s)
        D3[i] = sqrt((V_s[i][1]-v_0[1])^2 + (V_s[i][2]-v_0[2])^2)
    end

    return D1,D2,D3
end

function heuristique_1(D1, D2)
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations

    # On initialise le vecteur liant turbines et sous-stations
    L = zeros(length(D1),1)

    # On initialise le vecteur des sous-stations
    S = zeros(length(D2),1)

    # On emplit ce vecteur en associant à chaque turbine la sous-station la plus proche
    for i in 1:length(D1)
        L[i] = argmin(D1[i,:])
        S[L[i]][1] += 1
    end

    # On emplit ce vecteur en associant à chaque sous-station présente dans L la sous-station la plus proche
    for i in 1:length(L)
        if S[L[i]][2] == 0
            S[L[i]][2] = argmin(D2[L[i],:])
            S[S[L[i]][2]][2] = L[i]
        end
    end

    return L,S
    
end

function voisinage_turbine_aux_1(L, S, D1, D2, D3)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une turbine, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    L1 = L
    S1 = S

    new = rand(1:length(L))

    old = L1[new] 
    L1[new] = argmin(D2[L1[new],:])
    S1[old][1] -= 1
    if S1[old][1] == 0 && S1[S1[old][2]][1] ==0
        S1[S[old][2]][2] = 0
        S1[old][2] = 0
    end

    S[L1[new]][1] += 1
    if S1[L1[new]][2] == 0 && S1[L1[argmin(D1[L1[new],:])]][2] == 0
        S1[L1[new]][2] = argmin(D2[L1[new],:])
        S1[L1[argmin(D2[L1[new],:])]][2] = L1[new]
    end

    if cost(L1, S1, D1, D2, D3) < cost(L, S, D1, D2, D3)
        return L1, S1
    else
        return L, S
    end
end

function voisinage_substation_aux_1(L, S, D1, D2, D3)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une sous-station reliée à une turbine, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    L1 = L
    S1 = S

    new = rand(1:length(L))

    new = L1[new]

    old = S1[new][2]
    S1[old][2] = 0

    if S1[argmin(D2[old,:])][2]==0 && S1[argmin(D2[old,:])][1]!=0
        S1[argmin(D2[old,:])][2] = new
        S1[new][2] = argmin(D2[old,:])
    else
        S1[new][2]=0
    end
    
    if cost(L1, S1, D1, D2, D3) < cost(L, S, D1, D2, D3)
        return L1, S1
    else
        return L, S
    end
end

function proche_voisinage(L, S, D1, D2, D3)
    for i in 1000
        L,S = voisinage_turbine_aux_1(L, S, D1, D2, D3)
        L,S = voisinage_substation_aux_1(L, S, D1, D2, D3)
    end
    return L,S
end


        







proche_voisinage (generic function with 1 method)

In [11]:
#collectes des données utiles

file = json_small

P_max = get_parameters(file)["maximum_power"]
coords_mls = get_parameters(file)["main_land_station"]
C_ft = get_parameters(file)["fixed_cost_cable"]
C_lt = get_parameters(file)["variable_cost_cable"]
C_0 = get_parameters(file)["curtailing_cost"]
C_p = get_parameters(file)["curtailing_penalty"]
C_max = get_parameters(file)["maximum_curtailing"]

wind_turbines = [get_wind_turbines(file)]

wind_scenarios = get_wind_scenarios(file)

substation_locations = get_substation_locations(file)

substation_types = get_substation_types(file)

land_substation_cable_types = get_land_substation_cable_types(file)

sub_sub_cable_types = get_substation_substation_cable_types(file)


UndefVarError: UndefVarError: json_small not defined